# Supplementary File 
# Inferring Table 1 Parameter Values 

This notebook describes how the numerical parameter estimates in Table 1 were inferred from the data sources described in Table 1. For consistency throughout up to four significant digits are displayed and time units are scaled as 1/90 yrs as per the code in http://github.com/kewok/wanipva. The tables "fish_alpha.txt" and "MedemFood1C_PrimaryVert.txt" can be obtained from the code repository. Additional parameter values are reported directly in the references and so are fixed as global variables here.

In [3]:
significant_digits <- 4
intra_step_time_steps <- 90

## Directly reported parameter values

In [4]:
maximum_weight <- 58000 # Ojasti 1996
p_epsilon <- 0.225 # Ojasti 1996
max_age <- 40 # Weigl 2014
L_e <- 3.0 # Saalfeld et al. 2008
mu <- 0.75 # West et al. 2017

## Parameters related to somatic growth without resource scarcity: $c_e, \gamma, m_c$, and the length-weight conversion terms.

Parameters related to somatic growth are inferred under *ad libitum* feeding in Perez-Talavera

In [5]:

library(minpack.lm)
library(FME)
wt <-  c(38.2,150,300,600,1100,1400,1900,2300,2500,3100,maximum_weight) # Weight at month (based on mean values in Perez Talavera collected via datathief; Fig. 2)
times <- c(seq(30,300,30)*(intra_step_time_steps/365),40*intra_step_time_steps) # first ten months data from Perez Talavera and per Weigl 2014 assume maximum age of 40 years (so units are days)

xstart <- c(W=38.2)

grow <- function(t, x, parms)	{
	with(as.list(c(parms, x)), {
	c_e <- parms$c_e
	m_c <- parms$m_c
	g1 <- parms$g1
	g2 <- mu

	dW <- c_e*W^g1-m_c*W^g2
	list(dW)
	})
	}

pred <- function(parms)
	{
	c_e <- parms[1]
	m_c <- parms[2]
	g1 <- parms[3]
	g2 <- mu
	out <- lsoda(xstart, times, grow, parms=list(c_e=c_e,m_c=m_c,g1=g1,g2=g2),maxsteps=2e5)
	return(out)
	}

# FME version

rss <- function(parms)
	{
	data <- cbind(times, log(wt))
	colnames(data) <- c('time','W')

	c_e <- parms[1]
	m_c <- parms[2]
	g1 <- parms[3]
	g2 <- mu 

	out <- lsoda(xstart, times, grow, parms=list(c_e=c_e,m_c=m_c,g1=g1,g2=g2),maxsteps=2e5,atol=.Machine$double.eps)
	# Ensure that the predicted function is always increasing; otherwise return garbage
	if (is.na(out[2,2]))
		{
		out[2:nrow(out),2] <- rep(1e-10,nrow(out)-1)
		}
	if (out[1,2] > out[2,2])
		{
		out[2:nrow(out),2] <- rep(1e-10,nrow(out)-1)
		}
	out[,2] <- log(out[,2])
	return(modCost(obs=data, model=out))
	}

Loading required package: deSolve

Loading required package: rootSolve

Loading required package: coda



Initial guess

In [6]:

f <- function(c_e,m_c,g1)
	{
	fitval$par['c_e']=c_e
	fitval$par['m_c']=m_c
	fitval$par['g1']=g1
	}
#e,g,m f(2,0.5,.75)
parms <- c(c_e=2,m_c=0.5,g1=0.75)

fitval <- modFit(p=parms, f=rss, lower=c(1e-6,1e-6,0.5),upper=c(20,20,5))
cbind(lsoda(xstart,times,grow, as.list(fitval$par),maxsteps=2e5), wt)
print("Somatic growth and metabolism parameters:")
print(signif(fitval$par,significant_digits))

time,W,wt
7.39726,38.2000,38.2
14.79452,152.5658,150.0
22.19178,343.6423,300.0
29.58904,606.2972,600.0
36.98630,934.2701,1100.0
44.38356,1321.2240,1400.0
51.78082,1761.0994,1900.0
59.17808,2248.2366,2300.0
66.57534,2777.4084,2500.0
73.97260,3343.8173,3100.0


[1] "Somatic growth and metabolism parameters:"
   c_e    m_c     g1 
1.7140 0.3541 0.6062 


## Inferring the juvenile and adult maximum conditions $q_J$ and $q_A$

Using data in Congdon et al. 1995, $\max$(lipid)/$\min$(protein + ash mass) =

In [7]:
q_J <- 3.29/(3.9+0.56) # probably on the lower end (however, quite comparable to Persson 1998)

From Thorjarnarson 1994, relative clutch mass of 0.15 at maximum (pg 915 table 8) so simplify: 

$((x+q_A x)-(x+q_J x))/(x + q_A x) = (q_A x-q_J x)/(x+q_A x)$

$\rightarrow 0.15 = (q_A-q_J)/(1+q_A)$

$\rightarrow qA = (-q_J/.15-1)/(1-1/.15)$

And thus

In [8]:

q_A <- (-q_J/.15-1)/(1-1/.15)
print("Juvenile and adult maximum condition:")
print(signif(c(q_J,q_A),significant_digits))

[1] "Juvenile and adult maximum condition:"
[1] 0.7377 1.0440


## Conversion scalar $L_s$ of irreversible weight to length 

Total length (cm) and weight at month (g); tanque 1 from Perez-Talavera 2000 Fig. 1

In [9]:
tls <- c(22.5,32,44,56,64,72,80,84,88,96)
wt <-  c(38.2,150,300,600,1100,1400,1900,2300,2500,3100)

Since these are all TL < 120cm (the female size at maturation from Thorbjarnarson 94), assuming maximum condition for juveniles that wt in terms of irreversible mass are:


In [10]:

wtX <- wt/(1+q_J)

wtf <- wtX^(1/L_e)
summary(lm(tls~wtf))
print("Length weight regression coefficient")
print(signif(summary(lm(tls~wtf))$coefficients[2,1],significant_digits))
# So to convert total length to weight, coefficient of wtX^(1/3) is 7.814; note constant is 0 (p=0.73)
print("In decimal form")
print(signif(1/(lm(tls~wtf)$coefficients[2]),significant_digits))



Call:
lm(formula = tls ~ wtf)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.5600 -0.9889  0.1777  1.1117  1.7651 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.5300     1.4845  -0.357     0.73    
wtf           7.8136     0.1693  46.160 5.36e-11 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.608 on 8 degrees of freedom
Multiple R-squared:  0.9963,	Adjusted R-squared:  0.9958 
F-statistic:  2131 on 1 and 8 DF,  p-value: 5.361e-11


[1] "Length weight regression coefficient"
[1] 7.814
[1] "In decimal form"
  wtf 
0.128 


## Parameters governing consumption in response to scarcity: $a,h$

Based on high protein diet, Staton et al. 1990 Table 5. daily consumption:


In [11]:
df2 <- c(0, 1,2,3,3,5,5,6,7)/7

Food intake (dry matter consumption on protein diet):

In [12]:
y <- c(0, 212,399,499,519,582,557,722,641)/722

zz <- nls(y ~ a*(df2)/(1+a*h*(df2)), start=list(a=3,h=1) )

summary(zz)
print("Consumption in response to scarcity parameters:")
print(signif(summary(zz)$coefficients[,1],significant_digits))
fu <- function(x) 3.208*x/(1+3.208*x*0.7567)

# Pseudo R-sq.
au <- lm(fu(df2)~y)
summary(au)



Formula: y ~ a * (df2)/(1 + a * h * (df2))

Parameters:
  Estimate Std. Error t value Pr(>|t|)    
a   3.2085     0.5275   6.083    5e-04 ***
h   0.7567     0.0825   9.172 3.77e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.05851 on 7 degrees of freedom

Number of iterations to convergence: 5 
Achieved convergence tolerance: 1.114e-06


[1] "Consumption in response to scarcity parameters:"
     a      h 
3.2080 0.7567 



Call:
lm(formula = fu(df2) ~ y)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.09381 -0.02756 -0.01622  0.03770  0.07089 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.02756    0.04506   0.612     0.56    
y            0.95881    0.06430  14.912 1.46e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.05685 on 7 degrees of freedom
Multiple R-squared:  0.9695,	Adjusted R-squared:  0.9651 
F-statistic: 222.4 on 1 and 7 DF,  p-value: 1.463e-06


## Sizes at maturation for males ($S_{m,m}$) and females ($S_{m,f}$)

In Thorbjarnarson 1994, pg. 909 mean size at maturation of females is 64 cm SVL. Using the likely conversion of SVL = 4 + 0.5 TL (based on regression of TL = c(165,150,143) and SVL = c(87,79,76) on p.915 and rounding up to the nearest full cm), 64 cm SVL is appx. 120 cm TL. Using the inverse of the weight-length regression above:

In [13]:
lw <- function(TL) (TL*(1/(7.8136)))^L_e

So the mean irreversible mass at maturation is:

In [14]:
fmat <- lw(120)

Assuming an individual at this weight is reproducing, this is Xmat=3622.348 g

Thorbjarnarson 94 also reports males to be physiologically mature at 75 cm SVL, so based on SVL=4 + 0.5 *TL this is 142 cm TL


In [15]:
mmat <- lw(142)

print("Sizes at maturation:")
print(signif(c(fmat,mmat),significant_digits))


[1] "Sizes at maturation:"
[1] 3622 6002


## Parameters governing prey group consumption: $\alpha_{\textrm{inv}}, \beta_{\textrm{inv}}$ and $p_f$

Read in the data from Medem 1981 Table 1C, but with individuals with both fish and tetrapods removed. These are records no. 381, 342, 116, 127,319 in Medem's Table 1C

In [16]:
data <- read.table("MedemFood1C_PrimaryVert.txt",header=T)


Implicit in this approach is that the probability of no invertebrates is inversely proportional to the fraction of the diet that consists of invertebrates. This is because if an individual consumes no invertebrates, then the fraction of their diet that consists of invertebrates is zero. Whereas, if an individual's diet consists exclusively of invertebrates, we assume we will find invertebrate remains. Now, since the probability of finding invertebrates is $1-\Pr$(no invertebrates), the above reasoning suggests that we can model the fraction of the diet that consists of invertebrates as $\Pr$(invertebrates) = 1-$\Pr$(no invertebrates) up to a constant of proportionality. 

In [17]:
lw <- function(TL) (TL*(1/(7.8136)))^3
size_x <- lw(data[,'Length'])

ons_model <- glm(data[,'Invertebrate']~size_x, family=binomial(link="logit"))
summary(ons_model)
print("Invertebrate consumption parameters:")
print(signif(summary(ons_model)$coefficients[,1],significant_digits))
sizes <- seq(min(lw(data[,1])),max(lw(data[,1])),length=50)

preds <- predict(ons_model, list(size_x=sizes), type="response")


Call:
glm(formula = data[, "Invertebrate"] ~ size_x, family = binomial(link = "logit"))

Coefficients:
              Estimate Std. Error z value Pr(>|z|)    
(Intercept)  1.2651733  0.5119695   2.471 0.013466 *  
size_x      -0.0021536  0.0005652  -3.811 0.000139 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 94.607  on 77  degrees of freedom
Residual deviance: 63.552  on 76  degrees of freedom
AIC: 67.552

Number of Fisher Scoring iterations: 6


[1] "Invertebrate consumption parameters:"
(Intercept)      size_x 
   1.265000   -0.002154 


For remaining prey, first, remove the cases where there were no other prey items:

In [18]:
subset <- which(data[,"Vertebrate"] != "N")
print("Proportion of vertebrate diet that is fish:")
print(signif(table(data[subset,"Vertebrate"])[1]/length(subset),significant_digits))

[1] "Proportion of vertebrate diet that is fish:"
     F 
0.7826 


Thus, we can assume that 36/46 of the vertebrate diet is fish, and a 1-36/46 of the vertebrate diet is non-fish semiaquatic organisms.


## Neonate irreversible mass $\epsilon$

In [19]:
print("Neonate irreversible mass:")
print(signif(38.2/(q_J+1), significant_digits))

[1] "Neonate irreversible mass:"
[1] 21.98


## Parameters related to mortality: $\alpha_m, \beta_m, b_{s_0}$ and $b_s$.

 The equation for starvation induced mortality can be calculated as follows. Assume that at maximum condition 1e-10 die and if the starvation condition is reached 1-1e-10 die after 4 months (the duration of Garnett 1986's experiment). Note everything is converted to the scaled daily rate. Thus:


In [20]:
pS1 <- c(q_J, (1e-10))

 Per Garnett 1986, individuals who lose 27.4% of their body weight died of starvation after four months. So let their daily survivorship be $(1e-10)^{(1/(30*4)})$

In [21]:
starv_loss <- 0.274

 Therefore, if an individual at the juvenile maximum condition has weight $(1+q_J)x$, then at starvation they will have weight (1+$q_J$)(1-starv_loss). Thus the ratio of reversible to irreversible mass at this point is:

In [22]:
starv_ratio <- (1+q_J)*(1-starv_loss)-1

where -1 is to account for irreversible mass. This ensures starv_ration + starv_loss = 1. Now,

In [23]:
pS2 <- c(starv_ratio, (1-1e-10))


Thus from the equality of slopes, $(y-y2)/(x-x2) = (y1-y2)/(x1-x2)$

or

$\rightarrow (y-log(0.9999))/(x-log$(starv_ratio)$) = (log(0.0001)-log(0.9999))/(log(q_J)-log$(starv_ratio)$)$

$\rightarrow (y-log(0.9999))= (log(0.0001)-log(0.9999))/(log(q_J)-log$(starv_ratio)$)*(x-log$(starv_ratio)$)$

$\rightarrow y = (log(0.0001)-log(0.9999))/(log(q_J)-log$(starv_ratio)$)*(x-log$(starv_ratio)$)+log(0.9999)$

In [24]:
y <- function(x) (exp((log(pS1[2])-log(pS2[2]))/(log(pS1[1])-log(pS2[1]))*(log(x)-log(pS2[1]))+log(pS2[2])))

 This may seem generous, but accords with Garnett 1986's discussion that crocodilians appear to be able to go very long periods without starvation-induced effects. Thus the coefficients are 

In [25]:
bs <- (log(pS1[2])-log(pS2[2]))/(log(pS1[1])-log(pS2[1]))
bs0 <- bs*(-log(pS2[1]))+log(pS2[2])
print("Starvation equation constant & coefficient:")
print(signif(c(bs0,bs),significant_digits))

[1] "Starvation equation constant & coefficient:"
[1] -29.78 -22.21


 Size-dependent survivorship from other sources, esp. predation, can be estimated using the formula for logistic regression using first year survivorships based on Webb and Smith (Table 1 pg. 542)

In [26]:
wtX <- c(38.2/(q_J+1),lw((72))) # Irreversible mass at birth and consider individuals with SVL >= 40cm, who are likely to suffer very low predation. (e.g., Ayarzaguena 1983, "Las babas que superan el tamano medio de la Clase II (Lcc 400mm) puede suponerse que no tienen actualmente depradores naturales en el area de El Frio. (pg. 119)... Pasado este tiempo las babas alcanzan un talla proximo a medio metro (i.e., 50cm) que imposibilita mucho la predacion, aunque los peligros no desaparecen hasta un ano mas tarde (p. 122)")  Using the regression SVL = 4 + 0.5 TL from above, SVL of 40cm=TL of 72cm, which translates into a weight of lw(72)
survs <- c((1-.88)^(1/intra_step_time_steps), (1-1e-10)) # Assume that individuals at the size >40 cm SVL only suffer baseline (Weigl 2014) mortality 
p1 <- c(wtX[1],survs[1])
p2 <- c(wtX[2],survs[2]) 

Using the logit transform of the logistic function $\frac{1}{(1+\exp(-(\alpha_m+\beta_m x)))}$ (i.e., log(y/(1-y))) we get the system of equations:

$\alpha_m + \beta_m*p1[1]=log(p1[2]/(1-p1[2]))$

and 
 
$\alpha_m + \beta_m*p2[1]=log(p2[2]/(1-p2[2]))$

Solving for $\alpha_m,\beta_m$ we get:

$\alpha_m=log(p1[2]/(1-p1[2]))-\beta_m*p1[1]$

$ log(p1[2]/(1-p1[2]))-\beta_m*p1[1] +\beta_m*p2[1] = log(p2[2]/(1-p2[2]))$

$ \beta_m*(p2[1]-p1[1])=log(p2[2]/(1-p2[2]))-log(p1[2]/(1-p1[2]))$

and thus:

In [27]:
surv_coef <- (log(p2[2]/(1-p2[2]))-log(p1[2]/(1-p1[2])))/(p2[1]-p1[1])
surv_const <- log(p1[2]/(1-p1[2]))-(log(p2[2]/(1-p2[2]))-log(p1[2]/(1-p1[2])))/(p2[1]-p1[1])*p1[1]
print("Size specific survivorship constant and coefficient")
print(signif(c(surv_coef, surv_const),significant_digits))

[1] "Size specific survivorship constant and coefficient"
[1] 0.02537 3.17900


## Parameters related to recruitment of prey groups $r_{\textrm{fish}}, K_{\textrm{fish}}:K_{\textrm{tetr}}$ and $r_{\textrm{tetr}}$  

If we model the consumable resource biomass as consisting of average-sized individuals of the prey group with average fractions of consumable biomass, the per-unit increase in average consumable biomass should be directly proportional, on average, to the population density-independent recruitment rate. The density-independent recruitment $r_{\textrm{fish}}$ of fish prey group biomass can therefore be inferred from Myers et al. 1999

In [28]:
fish <- read.table("fish_alpha.txt",header=TRUE)
print("Maximum fish growth rate:")
print(signif(median(fish[,1])^(1/intra_step_time_steps),significant_digits))

[1] "Maximum fish growth rate:"
[1] 1.026


The fish:tetrapod biomass ratio reported in Casagranda and Boudouresque 2009 yields 

In [29]:
print(signif( (1.3/(6.4+13.8)),significant_digits))

[1] 0.06436


From there, $r_{\textrm{tetr}}$ was determined as described in Table 1 of the main text. 